# 🐝 Aura Node: Interactive Worker

This notebook allows you to run an Aura Inference Node on Google Colab using Ollama and Gradio.

**Disclaimer:** For Research & Development Use Only. Do not use for commercial hosting.

In [ ]:
# @title 1. Install Dependencies
# NOTE: Piping curl to sh carries security risks. Review scripts before execution.
!curl -fsSL https://ollama.com/install.sh | sh
!pip install gradio requests toml

# Download and verify frpc
import hashlib
import os

FRPC_VERSION = "0.61.0"
FRPC_SHA256 = "720a9fe2a3299346572544909a78c023344c88bde13c55b921e298e8c5ded21f"
FRPC_FILE = f"frp_{FRPC_VERSION}_linux_amd64.tar.gz"
FRPC_URL = (
    f"https://github.com/fatedier/frp/releases/download/v{FRPC_VERSION}/{FRPC_FILE}"
)

if not os.path.exists("frpc"):
    print(f"Downloading frpc v{FRPC_VERSION}...")
    !curl -L {FRPC_URL} -o frp.tar.gz

    # Verify Checksum
    with open("frp.tar.gz", "rb") as f:
        file_hash = hashlib.sha256(f.read()).hexdigest()

    if file_hash != FRPC_SHA256:
        print(
            f"CRITICAL: Checksum verification failed! Expected {FRPC_SHA256}, got {file_hash}"
        )
        os.remove("frp.tar.gz")
    else:
        print("Checksum verified.")
        !tar -xzf frp.tar.gz
        !cp frp_{FRPC_VERSION}_linux_amd64/frpc .
        !chmod +x frpc
        !rm -rf frp.tar.gz frp_{FRPC_VERSION}_linux_amd64

if os.path.exists("frpc"):
    print("Setup complete!")
else:
    print("Setup failed.")

In [ ]:
# @title 2. Launch Aura Node UI
import collections
import os
import subprocess
import sys
import threading
import time

import gradio as gr
import requests
import toml


# Logging system to redirect stdout to Gradio
class Logger:
    def __init__(self):
        # Keep the last 1000 chunks of log data
        self.logs = collections.deque(maxlen=1000)
        self.lock = threading.Lock()

    def write(self, data):
        with self.lock:
            self.logs.append(data)

    def get_logs(self):
        with self.lock:
            return "".join(self.logs)


logger = Logger()


class StreamToLogger:
    def __init__(self, stream, logger):
        self.stream = stream
        self.logger = logger

    def write(self, data):
        self.stream.write(data)
        self.logger.write(data)

    def flush(self):
        self.stream.flush()


# Prevent duplicate wrapping
if not isinstance(sys.stdout, StreamToLogger):
    sys.stdout = StreamToLogger(sys.stdout, logger)
    sys.stderr = StreamToLogger(sys.stderr, logger)


class AuraNode:
    def __init__(self):
        self.ollama_process = None
        self.frpc_process = None
        self.status = "Idle"
        self.requests_processed = 0
        self.is_running = False
        self.lock = threading.Lock()

    def start(
        self,
        model,
        name,
        token,
        remote_port,
        server_addr,
        server_port,
        progress=None,
    ):
        if progress is None:
            progress = gr.Progress()
        with self.lock:
            if self.is_running:
                return "Already running"
            self.is_running = True
            self.status = "Processing"

        try:
            # 1. Start Ollama
            progress(0, desc="Starting Ollama server...")
            print("--- Starting Ollama server ---")
            if self.ollama_process is None:
                self.ollama_process = subprocess.Popen(
                    ["ollama", "serve"],
                    stdout=subprocess.PIPE,
                    stderr=subprocess.STDOUT,
                    text=True,
                )
                threading.Thread(
                    target=self._read_output, args=(self.ollama_process,), daemon=True
                ).start()

            # Wait for ollama to be up
            for i in range(30):
                try:
                    requests.get("http://localhost:11434")
                    break
                except requests.exceptions.RequestException:
                    time.sleep(1)
                    progress(i / 30, desc=f"Waiting for Ollama ({i}s)")
            else:
                self.stop()  # Cleanup
                with self.lock:
                    self.status = "Error: Ollama failed to start"
                return "Ollama failed to start"

            # 2. Pull model
            progress(0.1, desc=f"Pulling model {model}...")
            print(f"--- Pulling model: {model} ---")
            pull_proc = subprocess.Popen(
                ["ollama", "pull", model],
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                text=True,
            )
            for line in pull_proc.stdout:
                print(line, end="")
            pull_proc.wait()

            if pull_proc.returncode != 0:
                self.stop()  # Cleanup
                with self.lock:
                    self.status = f"Error: Failed to pull {model}"
                return f"Failed to pull {model}"

            # 3. Start frpc
            progress(0.9, desc="Connecting to Hive via frpc...")
            self._start_frpc(name, token, remote_port, server_addr, server_port)

            with self.lock:
                self.status = "Connected"
            print("--- Aura Node is now Connected to the Hive! ---")
            return "Node Connected!"

        except Exception as e:
            print(f"Error: {e}")
            self.stop()
            with self.lock:
                self.status = f"Error: {e}"
            return f"Error: {e}"

    def _start_frpc(self, name, token, remote_port, server_addr, server_port):
        config_data = {
            "serverAddr": server_addr,
            "serverPort": int(server_port),
            "auth": {"method": "token", "token": token},
            "proxies": [
                {
                    "name": name,
                    "type": "tcp",
                    "localIP": "127.0.0.1",
                    "localPort": 11434,
                    "remotePort": int(remote_port),
                }
            ],
        }

        with open("frpc.toml", "w") as f:
            toml.dump(config_data, f)

        print(f"--- Starting frpc tunnel for {name} on {server_addr}:{server_port} ---")
        if self.frpc_process:
            self.frpc_process.terminate()

        self.frpc_process = subprocess.Popen(
            ["./frpc", "-c", "frpc.toml"],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
        )
        threading.Thread(
            target=self._read_output, args=(self.frpc_process,), daemon=True
        ).start()

    def _read_output(self, process):
        for line in process.stdout:
            print(line, end="")
            # Increment stats on successful inference requests
            with self.lock:
                if "POST /api/generate" in line or "POST /api/chat" in line:
                    self.requests_processed += 1
                if "login to server success" in line:
                    self.status = "Connected"

    def stop(self):
        print("--- Stopping Aura Node ---")
        if self.frpc_process:
            self.frpc_process.terminate()
            self.frpc_process = None
        if self.ollama_process:
            self.ollama_process.terminate()
            self.ollama_process = None

        # Clean up config file to prevent token leakage
        if os.path.exists("frpc.toml"):
            try:
                os.remove("frpc.toml")
            except OSError:
                pass

        with self.lock:
            self.is_running = False
            self.status = "Idle"
        return "Node stopped"


node = AuraNode()

with gr.Blocks(title="Aura Node", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🐝 Aura Interactive Worker Node")
    gr.Markdown(
        "**Disclaimer:** For Research & Development Use Only. Do not use for commercial hosting."
    )

    with gr.Row():
        with gr.Column(scale=1):
            status_label = gr.Label(value=node.status, label="Brain Status")
            stats_box = gr.Number(
                value=node.requests_processed, label="Requests Processed"
            )
        with gr.Column(scale=2):
            with gr.Row():
                model_input = gr.Textbox(value="mistral", label="Ollama Model")
                worker_name_input = gr.Textbox(
                    value="aura-worker-colab", label="Worker Name"
                )
            with gr.Row():
                server_addr_input = gr.Textbox(
                    value="aura.zae.life", label="FRP Server Address"
                )
                server_port_input = gr.Textbox(value="7000", label="FRP Server Port")
            with gr.Row():
                remote_port_input = gr.Textbox(value="8083", label="Remote Port")
                token_input = gr.Textbox(label="Hive FRP Token", type="password")

    with gr.Row():
        start_btn = gr.Button("Start Node", variant="primary")
        stop_btn = gr.Button("Stop Node", variant="stop")

    log_output = gr.Textbox(lines=15, label="Agent Thinking / Logs", interactive=False)

    # Refresh logic for UI
    timer = gr.Timer(2)

    def refresh():
        with node.lock:
            if node.is_running:
                # Check if processes are still alive
                if node.ollama_process and node.ollama_process.poll() is not None:
                    node.status = "Error: Ollama process stopped"
                    node.is_running = False
                elif node.frpc_process and node.frpc_process.poll() is not None:
                    node.status = "Error: frpc process stopped"
                    node.is_running = False

            status = node.status
            requests_processed = node.requests_processed
        logs = logger.get_logs()
        return status, requests_processed, logs

    timer.tick(refresh, outputs=[status_label, stats_box, log_output])

    start_btn.click(
        node.start,
        inputs=[
            model_input,
            worker_name_input,
            token_input,
            remote_port_input,
            server_addr_input,
            server_port_input,
        ],
        outputs=[log_output],
    )
    stop_btn.click(node.stop, outputs=[log_output])

demo.launch(share=True, inline=False)